In [0]:
# Lab 12 Character Sequence RNN
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)  # reproducibility

In [0]:
sample = " if you want you"
idx2char = list(set(sample))  # index -> char
char2idx = {c: i for i, c in enumerate(idx2char)}  # char -> idex

In [0]:
# hyper parameters
dic_size = len(char2idx)  # RNN input size (one hot size)
hidden_size = len(char2idx)  # RNN output size
num_classes = len(char2idx)  # final output size (RNN or softmax, etc.)
batch_size = 1  # one sample data, one batch
sequence_length = len(sample) - 1  # number of lstm rollings (unit #)
learning_rate = 0.1

In [0]:
sample_idx = [char2idx[c] for c in sample]  # char to index
x_data = [sample_idx[:-1]]  # X data sample (0 ~ n-1) hello: hell
y_data = [sample_idx[1:]]   # Y label sample (1 ~ n) hello: ello

X = tf.placeholder(tf.int32, [None, sequence_length])  # X data
Y = tf.placeholder(tf.int32, [None, sequence_length])  # Y label

In [0]:
x_one_hot = tf.one_hot(X, num_classes)  # one hot: 1 -> 0 1 0 0 0 0 0 0 0 0
cell = tf.nn.rnn_cell.LSTMCell(
    num_units=hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(
    cell, x_one_hot, initial_state=initial_state, dtype=tf.float32)


In [0]:
# FC layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)

# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])


In [0]:
weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

prediction = tf.argmax(outputs, axis=2)

In [8]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(50):
        l, _ = sess.run([loss, train], feed_dict={X: x_data, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_data})

        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]

        print(i, "loss:", l, "Prediction:", ''.join(result_str))




0 loss: 2.2958248 Prediction:  y             
1 loss: 2.1152258 Prediction: yy  oo         
2 loss: 1.9109607 Prediction: yy you    yyyou
3 loss: 1.6330148 Prediction: y  youuu n  you
4 loss: 1.3691957 Prediction: y  youuw n  you
5 loss: 1.1307048 Prediction: y  youuwant you
6 loss: 0.8773393 Prediction: y  you want you
7 loss: 0.6543162 Prediction: yf you want you
8 loss: 0.48079887 Prediction: if you want you
9 loss: 0.33993512 Prediction: if you want you
10 loss: 0.23654513 Prediction: if you want you
11 loss: 0.1614703 Prediction: if you want you
12 loss: 0.10955138 Prediction: if you want you
13 loss: 0.07365052 Prediction: if you want you
14 loss: 0.050194997 Prediction: if you want you
15 loss: 0.035238467 Prediction: if you want you
16 loss: 0.025278052 Prediction: if you want you
17 loss: 0.018272284 Prediction: if you want you
18 loss: 0.013281718 Prediction: if you want you
19 loss: 0.009780107 Prediction: if you want you
20 loss: 0.0073490413 Prediction: if you want you
21 